In [3]:
import os
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [4]:
#selenium import
#https://greeksharifa.github.io/references/2020/10/30/python-selenium-usage/
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [10]:
#접속 및 로그인
#소요 시간: 6s

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

url = "https://www.dewey.org/webdewey/login/login.html"
driver.get(url=url)
driver.find_element(By.XPATH, '//*[@id="nameinput"]').send_keys('사용자ID')
driver.find_element(By.XPATH, '//*[@id="wrapper"]/div[1]/form/div/div[2]/div[2]/input').send_keys('사용자PASSWORD')
driver.find_element(By.XPATH, '//*[@id="wrapper"]/div[1]/form/div/div[3]/div[2]/button/div[2]/div[2]').click()

In [6]:
#크롤링
#주류(Main Class), 강목(Devision), 요목(Section)까지 가져오기
#소요 시간: 1m 15s
from tqdm import tqdm
base_url = 'https://www.dewey.org/webdewey/mainClasses.html?recordId=ddc%3a'

rows = [] #임시저장

for i in tqdm(range(0, 10), desc='Outer Loop'):
    for j in tqdm(range(0, 10), desc='Inner Loop'):
        url = base_url + str(i) + str(j)
        driver.get(url=url)
        
        content_container = driver.find_element(By.ID, 'content_container_padding')
        elements = content_container.find_elements(By.CLASS_NAME, 'clearafter')
        element_number = 0 #element 순서 저장.
        
        for element in elements:
            if(element_number == 0): #첫 번째 element는 주류
                main_class = element.text.split('\n')[1]
                devision = ''
            
            elif(element_number == 1): #두 번째 element는 강목
                devision = element.text.split('\n')[1]
                element_number += 1
                continue
            
            number, section = element.text.split('\n')
            rows.append({'주류': main_class, '강목': devision, '요목번호': number, '요목명': section})
            element_number += 1

driver.quit()
df = pd.DataFrame(rows)

#중복 행 삭제
df = df.drop_duplicates().reset_index(drop=True)

df.head(30)

Outer Loop:   0%|          | 0/10 [00:00<?, ?it/s]

Outer Loop: 100%|██████████| 10/10 [01:11<00:00,  7.16s/it]


,주류,강목,요목번호,요목명
0,"Computer science, information & general works",,000,"Computer science, information & general works"
1,"Computer science, information & general works","Computer science, knowledge & systems",000,"Computer science, information, general works"
2,"Computer science, information & general works","Computer science, knowledge & systems",001,Knowledge
3,"Computer science, information & general works","Computer science, knowledge & systems",002,The book
4,"Computer science, information & general works","Computer science, knowledge & systems",003,Systems
5,"Computer science, information & general works","Computer science, knowledge & systems",004-006,"Computer science; software development, softwa..."
6,"Computer science, information & general works","Computer science, knowledge & systems",[007],[Unassigned]
7,"Computer science, information & general works","Computer science, knowledge & systems",[008],[Never assigned]
8,"Computer science, information & general works","Computer science, knowledge & systems",[009],[Never assigned]
9,"Computer science, information & general works",Bibliographies,010,Bibliography


In [11]:
#크롤링
#세목(Subsection) 가져오기
#소요 시간: 6m

# '요목번호'에 대괄호가 없는 항목 필터링
filtered_df = df[~df['요목번호'].str.contains('\[.*\]')]

subsection_data = [] #새로운 df를 생성할 사전을 담는 리스트 생성

# 필터링된 데이터프레임의 각 요목번호에 대해 작업 수행
for index, row in tqdm(filtered_df.iterrows(), desc='df', total=filtered_df.shape[0]):
    url = base_url + row['요목번호']
    driver.get(url)
    
    # 필요한 요소를 찾기 (여기서는 가상의 클래스 'clearafter'를 사용)
    elements = driver.find_elements(By.CLASS_NAME, 'clearafter')
    element_number = 0

    # 각 요소에서 텍스트를 추출하여 저장
    for element in elements:
        if(element_number <= 1) :
            element_number += 1
            continue
        number, name = element.text.split('\n')

        subsection_data.append({
            '주류': row['주류'],
            '강목': row['강목'],
            '요목번호': row['요목번호'],
            '요목명': row['요목명'],
            '세목번호': number,
            '세목명': name
        })
       
        element_number += 1
        
subsection_df = pd.DataFrame(subsection_data)

df: 100%|██████████| 560/560 [06:24<00:00,  1.46it/s]


In [37]:
subsection_df.to_csv('dewey.csv')